<a href="https://colab.research.google.com/github/nlp2bok/fintech_/blob/lmh_dev/Data_Preprocessing_bond.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Data Preprocessing

##. 1. 채권분석 보고서(Bond Analysis Report)

1. 채권보고서 파일명(cName_rDate.txt)

 1495개의 채권보고서를 하나의 csv파일에 넣는다.
  파일명에서 cName, rDate추출, 파일 read를 통한 내용 추출('회사명','작성일자','내용')
2. 다양한 Tokenized 방식을 통해 전처리를 실행한다.

 Komoran, Hannanum,  Okt, Kkma, Mecab => Mecab사용
 
 -> Mecab install 필요
3. Token에 한글 태그를 부착한다.
4. 불용어 처리를 실행한다.
5. csv파일에 preprocessing data 컬럼을 추가한다(preData)





In [0]:
import pandas as pd
import os
import csv

##Step 1. 전체 파일을 하나의 파일로 합치기

In [0]:
#print(dirpath) #현재경로 출력
workDIr = os.path.abspath('/Users/minhyeon/Desktop/BondReport_Text/bText/')
cnt = 0

def fileopen(path):
    f = open(path,'r', encoding='utf-8')
    line = f.read()
    f.close()
    
    return line 

with open('BondFile.csv', 'w', encoding='utf-8', newline='') as cvsFile :
    wr = csv.writer(cvsFile)

    for dirpath, dirnames, filenames in os.walk(workDIr):
        for filename in filenames:
            #파일 컨텐츠 읽기
            path = dirpath+'/'+filename
            fType = filename[-4:]
            try:
                if fType == ".txt" : 
                    text = fileopen(path)
                else :
                    continue
                #파일 타입 체크
            except Exception as ex:
                print(filename + '파일 read 에러가발생 했습니다', ex)


            try:
                rDate = "20" + filename[-12:-4].replace('.','') #Report Date, String형
                cName = filename[:-13] #Company Name
                print(cnt)
                print(cName)
                cnt += 1
                wr.writerow([cName,rDate,text])
            except Exception as ex:
                print(filename + '에러가 발생 했습니다', ex)
    
cvsFile.close()




0
Daewoo
1
Daewoo
2
Daewoo
3
Hyundai
4
HanaFinance
5
Ebest
6
Ebest
7
Ebest
8
Hyundai
9
Daewoo
10
Daewoo
11
Daewoo
12
Daewoo
13
Daewoo
14
HanaFinance
15
Ebest
16
Ebest
17
Hyundai
18
Hana Daetoo
19
Ebest
20
Ebest
21
HanaFinance
22
Hyundai
23
Hyundai
24
Daewoo
25
Daewoo
26
Hyundai
27
Daewoo
28
Daewoo
29
HanaFinance
30
Daewoo
31
Daewoo
32
Hyundai
33
Daewoo
34
Daewoo
35
HanaFinance
36
Ebest
37
Ebest
38
Ebest
39
Ebest
40
Hyundai
41
Ebest
42
Ebest
43
Cape
44
Ebest
45
Daewoo
46
Daewoo
47
Daewoo
48
Hyundai
49
Daewoo
50
HanaFinance
51
Hyundai
52
Etrade
53
HanaFinance
54
HanaFinance
55
Hyundai
56
Hyundai
57
Daewoo
58
Daewoo
59
Daewoo
60
Hyundai
61
Daewoo
62
Daewoo
63
Daewoo
64
HanaFinance
65
Ebest
66
Ebest
67
HanaFinance
68
Hyundai
69
HanaFinance
70
Ebest
71
Daewoo
72
Daewoo
73
Hyundai
74
Daewoo
75
Daewoo
76
Daewoo
77
Daewoo
78
Ebest
79
Hyundai
80
HanaFinance
81
Ebest
82
Ebest
83
HanaFinance
84
Etrade
85
HanaFinance
86
Ebest
87
Ebest
88
Hyundai
89
Daewoo
90
Hyundai
91
Ebest
92
Ebest
93
Ebest
94
H

847
Hyundai
848
Etrade
849
Etrade
850
Cape
851
Ebest
852
HanaFinance
853
Hyundai
854
Ebest
855
Hyundai
856
Ebest
857
Cape
858
Hyundai
859
Ebest
860
HanaFinance
861
HanaFinance
862
Hyundai
863
Ebest
864
Daewoo
865
Daewoo
866
Etrade
867
Hyundai
868
Daewoo
869
Daewoo
870
HanaFinance
871
HanaFinance
872
Ebest
873
Ebest
874
Ebest
875
Hanhwa
876
Ebest
877
Daishin
878
HanaFinance
879
HanaFinance
880
Hyundai
881
Ebest
882
Hyundai
883
Daewoo
884
Hyundai
885
Daewoo
886
Hyundai
887
Daewoo
888
Daewoo
889
Ebest
890
Ebest
891
HanaFinance
892
Ebest
893
Ebest
894
Ebest
895
Ebest
896
Ebest
897
Daishin
898
Hyundai
899
Hyundai
900
Daewoo
901
Ebest
902
Ebest
903
Ebest
904
Ebest
905
Etrade
906
Hyundai
907
Hyundai
908
Daewoo
909
Daewoo
910
Hyundai
911
Hyundai
912
Hyundai
913
Hyundai
914
Daewoo
915
Daewoo
916
Hyundai
917
Daewoo
918
Hyundai
919
Daewoo
920
Daewoo
921
Hyundai
922
Hyundai
923
Hyundai
924
Daewoo
925
Hyundai
926
Hyundai
927
Daewoo
928
Hyundai
929
HanaFinance
930
HanaFinance
931
Hyundai
932
Hyundai

## Step2. Tokenized 실행

##2-1. 파일 읽어오기

In [0]:
#csv파일을 읽어오고 column명을 설정해준다.
mydata = pd.read_csv("BondFile.csv",names=['cName','rDate','bContent'],encoding="utf-8")

In [0]:
#읽어온 데이터를 DataFrame형식으로 담은 후 , mycontent에 전처리할 데이터만 담는다.
df = pd.DataFrame(mydata)
mycontent = df['bContent']

In [0]:
len(mycontent)
print(mydata.head())

         cName     rDate                                           bContent
0       Daewoo  20081218  Microsoft Word - 08년 Daewoo Bond Brief.docDaew...
1       Daewoo  20081230  <4D6963726F736F667420576F7264202D203039B3E2203...
2       Daewoo  20081224  Microsoft Word - 08년 Daewoo Bond Brief.docDaew...
3      Hyundai  20131219  Microsoft Word - HMC_FI_Comment_FOMC__2013.12....
4  HanaFinance  20171025  Microsoft Word - CrediVille_171025          당분...


## 2-2. 다양한 방식의 Tokenized 활용해보기


기사별 Tokenized한 값을 komoran_tokens, hannanum_tokens, okt_tokens, kkma_tokens, Mecab_tokens로 각각 저장한 후 결과를 확인해본다.

In [0]:
#출력을 위해서 임시 class 
class List(list): 
    def __str__(self): 
        return "[" + ", ".join(["%s" % x for x in self]) + "]"

In [0]:
#코모란(Komoran) 토큰화
from konlpy.tag import Komoran
komoran = Komoran()
komoran_tokens = []
for i in range(len(mycontent)):
    try:
        komoran_tokens.append(komoran.morphs(mycontent[i]))
        print(i)
    except:
        #이상 데이터 제외
        print(i)
        

0


In [0]:
#한나눔(Hannanum) 토큰화
from konlpy.tag import Hannanum
hannanum = Hannanum()
hannanum_tokens = []
for i in range(len(mycontent)):
    try:
        hannanum_tokens.append(hannanum.morphs(mycontent[i]))
    except:
        #이상 데이터 제외
        print(i)

In [0]:
#Okt 토큰화
from konlpy.tag import Okt
okt =Okt()
okt_tokens = []
for i in range(len(mycontent)):
    try:
        okt_tokens.append(okt.morphs(mycontent[i]))
    except:
        #이상 데이터 제외
        print(i)

In [0]:
#Kkma 토큰화
from konlpy.tag import Kkma
kkma = Kkma()
kkma_tokens = []
for i in range(len(mycontent)):
    try:
        kkma_tokens.append(kkma.morphs(mycontent[i]))
    except:
        #이상 데이터 제외
        print(i)

In [0]:
#MeCab 토큰화
from ekonlpy.tag import Mecab
mecab = Mecab()
mecab_tokens = []
for i in range(len(mycontent)):
    try:
        #print(mecab.pos(mycontent[i]))
        #break
        mecab_tokens.append(mecab.morphs(mycontent[i]))
        #print(i)
    except:
        #이상 데이터 제외
        print(i, sep='')
        print('error')

323
error
410
error
870
error


In [0]:
#토큰화된 데이터 중 첫번째 데이터 확인
print(mecab_tokens[412])

##Step3. Mecab을 사용하여 한글 태그 부착
가장 성능이 좋은 Mecab을 활용하기로 하였음.

한글 태그 부착은 .pos를 사용

In [0]:
#MeCab 토큰화(ekonlpy를 통한 한극 태그 부착 -> ekonlpy 설치 필요)
from ekonlpy.sentiment import Mecab
mecab = Mecab()
mecab_tokens = []
for i in range(len(mycontent)):
    try:
        mecab_tokens.append(mecab.pos(mycontent[i]))
        print(i)
    except:
        #이상 데이터 제외
        print('error')

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [0]:
#한글태그 부착확인
print(mecab_tokens[0])

##Step4. 불용어 처리
(1) stopPos에서는 불필요한 형태소를 한번에 제거

(2) stopWord는 불필요한 단어를 수동으로 제거

mecab-ko-dic 품사태그

부호 : SF(마침표, 물음표, 느낌표), SE(줄임표...), SSO(여는 괄호), SSC(닫는 괄호), SC(구분자),SY(띄어쓰기)

한글 이외 : SL(외국어), SH(한자), SN(숫자)


In [0]:
print(mecab_tokens[0])

[('Microsoft', 'SL'), ('Word', 'SL'), ('-', 'SY'), ('08', 'SN'), ('년', 'NNBC'), ('Daewoo', 'SL'), ('Bond', 'SL'), ('Brief', 'SL'), ('.', 'SY'), ('docDaewoo', 'SL'), ('Daewoo', 'SL'), ('Bond', 'SL'), ('Brief', 'SL'), ('채권', 'NNG'), ('윤일광', 'NNP'), ('(', 'SSO'), ('02', 'SN'), ('-', 'SY'), ('768', 'SN'), ('-', 'SY'), ('3058', 'SN'), (')', 'SSC'), ('ikyoon', 'SL'), ('@', 'SY'), ('bestez', 'SL'), ('.', 'SY'), ('com', 'SL'), ('2008', 'SN'), ('.', 'SY'), ('12', 'SN'), ('.', 'SY'), ('18', 'SN'), ('.', 'SF'), ('Thu', 'SL'), ('채권', 'NNG'), ('시황', 'NNG'), ('국고채', 'NNG'), ('년물', 'NNBC'), ('금리', 'NNG'), ('는', 'JX'), ('1', 'SN'), ('bp', 'SL'), ('하락', 'NNG'), ('한', 'XSV'), ('3', 'SN'), ('.', 'SY'), ('84', 'SN'), ('%,', 'SY'), ('년물', 'NNBC'), ('금리', 'NNG'), ('는', 'JX'), ('전일', 'NNG'), ('과', 'JC'), ('동일', 'NNG'), ('한', 'XSA'), ('4', 'SN'), ('.', 'SY'), ('74', 'SN'), ('%', 'SY'), ('로', 'JKB'), ('마감', 'NNG'), ('미연준', 'NNG'), ('의', 'JKG'), ('대폭', 'NNG'), ('적', 'XSN'), ('인', 'VCP'), ('통화', 'NNG'), ('완화정책',

In [0]:
#불용어 처리
stopPos = ['SF','SE','SSO','SSC','SC','SY','SL','SH','SN']
word = []
for doc in mecab_tokens :
    for mecab_token in doc:
        if mecab_token[1] not in stopPos:
            word.append(mecab_token)

In [0]:
print(word[0][0])

년


In [0]:
#불용어 처리
stopPos = ['SF','SE','SSO','SSC','SC','SY','SL','SH','SN']
stopWord = ["Microsoft"]
word_prepos = []
for doc in mecab_tokens : 
    for mecab_token in doc :
        if mecab_token[1] not in stopPos:
            if mecab_token[0] not in stopWord:
                word.append(mecab_token)
            

In [0]:
word

[('년', 'NNBC'),
 ('채권', 'NNG'),
 ('윤일광', 'NNP'),
 ('채권', 'NNG'),
 ('시황', 'NNG'),
 ('국고채', 'NNG'),
 ('년물', 'NNBC'),
 ('금리', 'NNG'),
 ('는', 'JX'),
 ('하락', 'NNG'),
 ('한', 'XSV'),
 ('년물', 'NNBC'),
 ('금리', 'NNG'),
 ('는', 'JX'),
 ('전일', 'NNG'),
 ('과', 'JC'),
 ('동일', 'NNG'),
 ('한', 'XSA'),
 ('로', 'JKB'),
 ('마감', 'NNG'),
 ('미연준', 'NNG'),
 ('의', 'JKG'),
 ('대폭', 'NNG'),
 ('적', 'XSN'),
 ('인', 'VCP'),
 ('통화', 'NNG'),
 ('완화정책', 'NNG'),
 ('에', 'JKB'),
 ('힘입', 'VV'),
 ('어', 'EC'),
 ('국내', 'NNG'),
 ('채권시장', 'NNG'),
 ('은', 'JX'),
 ('강세', 'NNG'),
 ('로', 'JKB'),
 ('출발', 'NNG'),
 ('국내외', 'NNG'),
 ('금리차', 'NNG'),
 ('확대', 'NNG'),
 ('로', 'JKB'),
 ('기준금리', 'NNG'),
 ('가', 'JKS'),
 ('추가', 'NNG'),
 ('하락', 'NNG'),
 ('할', 'XSV'),
 ('룸', 'NNG'),
 ('이', 'JKS'),
 ('커졌', 'VV'),
 ('고', 'EC'),
 ('상황', 'NNG'),
 ('이', 'JKS'),
 ('악화', 'NNG'),
 ('되', 'XSV'),
 ('면', 'EC'),
 ('결국', 'MAG'),
 ('한은', 'NNG'),
 ('도', 'JX'),
 ('이', 'NP'),
 ('를', 'JKO'),
 ('따라갈', 'VV'),
 ('수', 'NNB'),
 ('밖에', 'JX'),
 ('없', 'VA'),
 ('다는', 'ETM'),
 ('

##Step5. 전처리된 데이터를 기존 파일에 추가한다.